In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def calculate_smd(df, columns, treatment_col, weight_col=None):
    """
    Calculate Standardized Mean Differences (SMD) for a list of columns.
    
    Args:
    - df: DataFrame containing the data.
    - columns: List of column names to calculate SMD for.
    - treatment_col: Column name for the treatment indicator (1 for treated, 0 for control).
    - weight_col: Column name for the weights. If None, unweighted SMD will be calculated.
    
    Returns:
    - A DataFrame with the SMDs.
    """
    smds = []
    for column in columns:
        if weight_col:
            treated_mean = np.average(df.loc[df[treatment_col] == 1, column], weights=df.loc[df[treatment_col] == 1, weight_col])
            control_mean = np.average(df.loc[df[treatment_col] == 0, column], weights=df.loc[df[treatment_col] == 0, weight_col])
        else:
            treated_mean = df.loc[df[treatment_col] == 1, column].mean()
            control_mean = df.loc[df[treatment_col] == 0, column].mean()
        
        pooled_std = np.sqrt(((df.loc[df[treatment_col] == 1, column].var() + df.loc[df[treatment_col] == 0, column].var()) / 2))
        smd = (treated_mean - control_mean) / pooled_std
        smds.append(smd)
    
    return pd.DataFrame({'Variable': columns, 'SMD': smds})

def plot_love(df_pre, df_post, title='Love Plot for Covariate Balance'):
    """
    Generate a Love plot using pre- and post-weighting SMDs.
    
    Args:
    - df_pre: DataFrame with pre-weighting SMDs.
    - df_post: DataFrame with post-weighting SMDs.
    - title: Title of the plot.
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.scatter(df_pre['SMD'], df_pre['Variable'], label='Pre-weighting', color='red')
    ax.scatter(df_post['SMD'], df_post['Variable'], label='Post-weighting', color='blue')
    ax.axvline(x=0.1, color='grey', linestyle='--', label='SMD = 0.1 threshold')
    ax.axvline(x=-0.1, color='grey', linestyle='--')
    ax.set_xlabel('Standardized Mean Difference (SMD)')
    ax.set_title(title)
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:

# Assuming 'df' is your DataFrame, 'treatment' is the treatment indicator column,
# and 'weights' is the weights column. Replace 'covariates' with your list of covariate column names.
covariates = ['covariate1', 'covariate2']  # Example covariate column names
df_pre_smd = calculate_smd(df, covariates, 'treatment')
df_post_smd = calculate_smd(df, covariates, 'treatment', 'weights')

plot_love(df_pre_smd, df_post_smd)
